In [1]:
import os
import pickle as pkl

import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

Este dataframe no contiene ni las 5 muestras de carcinoma adenoescamoso, ni KoreanReference, ni WU-ref.

In [2]:
data = pd.read_csv('abundance_gene_MD_order.tsv', sep='\t')
data = data.rename(columns={'ProteinID': 'Sample ID'})
data

,Index,NumberPSM,Sample ID,MaxPepProb,ReferenceIntensity,CPT0237840004,CPT0166760004,CPT0094630004,CPT0063970004,CPT0081260004,...,CPT0078180003,CPT0197440004,CPT0355180003,CPT0355200003,CPT0355190003,CPT0347760002,CPT0347790002,CPT0347820002,CPT0347850002,CPT0347880002
0,A1BG,1395,P04217,1.0,28.716163,28.499551,28.452195,28.630157,29.174342,29.585927,...,28.496110,30.102289,29.425028,29.418836,28.214312,28.488643,28.385020,27.729956,28.458926,27.808411
1,A1CF,247,Q9NQ94,1.0,24.592863,24.018242,23.500280,24.174142,23.972000,24.501145,...,26.447539,23.165488,24.149451,25.233385,25.799610,25.981225,25.599940,25.914578,25.671271,26.218836
2,A2M,7832,P01023,1.0,29.941210,29.373080,29.038021,29.764657,30.442977,29.902549,...,29.658957,31.312727,30.928961,30.305615,29.207896,30.546957,29.339427,28.685161,29.364176,29.249406
3,A2ML1,141,A8K2U0,1.0,21.881356,NaN,NaN,NaN,NaN,NaN,...,22.009275,21.229600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A4GALT,3,Q9NPC4,1.0,16.076463,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12115,ZXDC,3,Q2QGD7,1.0,15.456372,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12116,ZYG11B,67,Q9C0D3,1.0,20.805448,20.918340,20.842082,20.881666,20.944515,21.111598,...,21.109123,20.985639,20.867963,20.997213,20.730738,20.547531,20.861241,20.549092,20.461376,20.356947
12117,ZYX,1854,Q15942,1.0,28.865534,28.917052,28.813093,28.692954,29.264926,29.331659,...,28.397063,28.819882,29.257749,28.014804,27.562795,27.765326,28.010625,27.360692,27.641424,27.304978
12118,ZZEF1,760,O43149,1.0,24.928770,25.079604,24.908095,24.953850,24.950274,24.707046,...,24.800248,24.973129,24.794404,24.702706,25.008415,24.978352,24.781041,25.387207,25.235190,25.447507


In [5]:
total_missing_percentage = (data.isnull().sum().sum() / (data.shape[0] * data.shape[1])) * 100
total_missing_percentage

22.19813717773451

In [56]:
# Hay varias proteínas que tienen dos códigos. Aquí estoy dejando únicamente un código

data['Sample ID'] = data['Sample ID'].replace('O43687;Q9P0M2', 'O43687')
data['Sample ID'] = data['Sample ID'].replace('O95467;P63092;Q5JWF2', 'O95467')
data['Sample ID'] = data['Sample ID'].replace('P0DPB5;P0DPB6', 'P0DPB5')
data['Sample ID'] = data['Sample ID'].replace('O00241;Q5TFQ8', 'O00241')
data['Sample ID'] = data['Sample ID'].replace('P42771;Q8N726', 'P42771')
data['Sample ID'] = data['Sample ID'].replace('O94854;Q9UPN3', 'O94854')
data['Sample ID'] = data['Sample ID'].replace('P0CAP2;Q6EEV4', 'P0CAP2')
data['Sample ID'] = data['Sample ID'].replace('P42166;P42167', 'P42166')
data['Sample ID'] = data['Sample ID'].replace('P39880;Q13948', 'P39880')
data['Sample ID'] = data['Sample ID'].replace('L0R8F8;Q9NQG6', 'L0R8F8')
data['Sample ID'] = data['Sample ID'].replace('B7ZAP0;Q5R372', 'B7ZAP0')
data['Sample ID'] = data['Sample ID'].replace('Q8NFQ8;Q9H496', 'Q8NFQ8')
data['Sample ID'] = data['Sample ID'].replace('P0DP91;Q03468', 'P0DP91')
data['Sample ID'] = data['Sample ID'].replace('O96007;O96033', 'O96007')
data['Sample ID'] = data['Sample ID'].replace('C0HLS1;Q14160', 'C0HLS1')

### MISSING VALUES IMPUTATION

In [57]:
columns_to_impute = data.columns.difference(['Sample ID', 'NumberPSM', 'Index', 'MaxPepProb', 'ReferenceIntensity'])

# Realiza la imputación solo en las columnas seleccionadas
imputer = KNNImputer(n_neighbors=5)
data[columns_to_impute] = imputer.fit_transform(data[columns_to_impute])
data

,Index,NumberPSM,Sample ID,MaxPepProb,ReferenceIntensity,CPT0237840004,CPT0166760004,CPT0094630004,CPT0063970004,CPT0081260004,...,CPT0078180003,CPT0197440004,CPT0355180003,CPT0355200003,CPT0355190003,CPT0347760002,CPT0347790002,CPT0347820002,CPT0347850002,CPT0347880002
0,A1BG,1395,P04217,1.0,28.716163,28.499551,28.452195,28.630157,29.174342,29.585927,...,28.496110,30.102289,29.425028,29.418836,28.214312,28.488643,28.385020,27.729956,28.458926,27.808411
1,A1CF,247,Q9NQ94,1.0,24.592863,24.018242,23.500280,24.174142,23.972000,24.501145,...,26.447539,23.165488,24.149451,25.233385,25.799610,25.981225,25.599940,25.914578,25.671271,26.218836
2,A2M,7832,P01023,1.0,29.941210,29.373080,29.038021,29.764657,30.442977,29.902549,...,29.658957,31.312727,30.928961,30.305615,29.207896,30.546957,29.339427,28.685161,29.364176,29.249406
3,A2ML1,141,A8K2U0,1.0,21.881356,21.264149,21.538211,21.404347,21.567005,21.898294,...,22.009275,21.229600,20.836430,21.697736,21.028232,20.980295,21.025700,20.925103,21.207741,21.143258
4,A4GALT,3,Q9NPC4,1.0,16.076463,17.221701,16.361668,16.615096,16.679397,17.552648,...,16.335277,16.372367,16.537346,16.196147,16.173505,16.310830,16.254974,16.348267,16.293014,16.530490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12115,ZXDC,3,Q2QGD7,1.0,15.456372,15.627144,15.481194,15.726177,15.928917,15.457205,...,15.273730,15.516053,15.838997,15.634068,15.526009,15.371021,15.388584,15.538631,15.627802,15.507224
12116,ZYG11B,67,Q9C0D3,1.0,20.805448,20.918340,20.842082,20.881666,20.944515,21.111598,...,21.109123,20.985639,20.867963,20.997213,20.730738,20.547531,20.861241,20.549092,20.461376,20.356947
12117,ZYX,1854,Q15942,1.0,28.865534,28.917052,28.813093,28.692954,29.264926,29.331659,...,28.397063,28.819882,29.257749,28.014804,27.562795,27.765326,28.010625,27.360692,27.641424,27.304978
12118,ZZEF1,760,O43149,1.0,24.928770,25.079604,24.908095,24.953850,24.950274,24.707046,...,24.800248,24.973129,24.794404,24.702706,25.008415,24.978352,24.781041,25.387207,25.235190,25.447507


In [58]:
data = data.set_index('Sample ID')

In [59]:
data = data.drop(columns=['NumberPSM', 'MaxPepProb', 'Index'])
data

,ReferenceIntensity,CPT0237840004,CPT0166760004,CPT0094630004,CPT0063970004,CPT0081260004,CPT0123990003,CPT0088840004,CPT0198420004,CPT0198910005,...,CPT0078180003,CPT0197440004,CPT0355180003,CPT0355200003,CPT0355190003,CPT0347760002,CPT0347790002,CPT0347820002,CPT0347850002,CPT0347880002
Sample ID,,,,,,,,,,,,,,,,,,,,,
P04217,28.716163,28.499551,28.452195,28.630157,29.174342,29.585927,28.764624,28.597662,28.876948,28.352665,...,28.496110,30.102289,29.425028,29.418836,28.214312,28.488643,28.385020,27.729956,28.458926,27.808411
Q9NQ94,24.592863,24.018242,23.500280,24.174142,23.972000,24.501145,24.182311,24.084523,23.584650,23.575798,...,26.447539,23.165488,24.149451,25.233385,25.799610,25.981225,25.599940,25.914578,25.671271,26.218836
P01023,29.941210,29.373080,29.038021,29.764657,30.442977,29.902549,30.134451,30.093668,30.854922,29.868366,...,29.658957,31.312727,30.928961,30.305615,29.207896,30.546957,29.339427,28.685161,29.364176,29.249406
A8K2U0,21.881356,21.264149,21.538211,21.404347,21.567005,21.898294,21.122291,21.232313,21.086050,21.337391,...,22.009275,21.229600,20.836430,21.697736,21.028232,20.980295,21.025700,20.925103,21.207741,21.143258
Q9NPC4,16.076463,17.221701,16.361668,16.615096,16.679397,17.552648,16.413556,16.725109,16.191675,16.289651,...,16.335277,16.372367,16.537346,16.196147,16.173505,16.310830,16.254974,16.348267,16.293014,16.530490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q2QGD7,15.456372,15.627144,15.481194,15.726177,15.928917,15.457205,15.527411,15.670059,15.073871,14.955910,...,15.273730,15.516053,15.838997,15.634068,15.526009,15.371021,15.388584,15.538631,15.627802,15.507224
Q9C0D3,20.805448,20.918340,20.842082,20.881666,20.944515,21.111598,20.888484,21.184368,20.642628,20.619563,...,21.109123,20.985639,20.867963,20.997213,20.730738,20.547531,20.861241,20.549092,20.461376,20.356947
Q15942,28.865534,28.917052,28.813093,28.692954,29.264926,29.331659,28.669558,28.872298,28.994721,28.989805,...,28.397063,28.819882,29.257749,28.014804,27.562795,27.765326,28.010625,27.360692,27.641424,27.304978


Separo la columna Reference Intensity para tenerla en otro dataframe para luego hacer las operaciones.

In [60]:
# Extraer la columna 'ReferenceIntensity' y colocarla en un nuevo DataFrame
reference_intensity_row = data[['ReferenceIntensity']].copy()

# Eliminar la columna 'ReferenceIntensity' del DataFrame original
data = data.drop(columns=['ReferenceIntensity'])
data

,CPT0237840004,CPT0166760004,CPT0094630004,CPT0063970004,CPT0081260004,CPT0123990003,CPT0088840004,CPT0198420004,CPT0198910005,CPT0166650006,...,CPT0078180003,CPT0197440004,CPT0355180003,CPT0355200003,CPT0355190003,CPT0347760002,CPT0347790002,CPT0347820002,CPT0347850002,CPT0347880002
Sample ID,,,,,,,,,,,,,,,,,,,,,
P04217,28.499551,28.452195,28.630157,29.174342,29.585927,28.764624,28.597662,28.876948,28.352665,28.808436,...,28.496110,30.102289,29.425028,29.418836,28.214312,28.488643,28.385020,27.729956,28.458926,27.808411
Q9NQ94,24.018242,23.500280,24.174142,23.972000,24.501145,24.182311,24.084523,23.584650,23.575798,24.302236,...,26.447539,23.165488,24.149451,25.233385,25.799610,25.981225,25.599940,25.914578,25.671271,26.218836
P01023,29.373080,29.038021,29.764657,30.442977,29.902549,30.134451,30.093668,30.854922,29.868366,29.865379,...,29.658957,31.312727,30.928961,30.305615,29.207896,30.546957,29.339427,28.685161,29.364176,29.249406
A8K2U0,21.264149,21.538211,21.404347,21.567005,21.898294,21.122291,21.232313,21.086050,21.337391,21.176486,...,22.009275,21.229600,20.836430,21.697736,21.028232,20.980295,21.025700,20.925103,21.207741,21.143258
Q9NPC4,17.221701,16.361668,16.615096,16.679397,17.552648,16.413556,16.725109,16.191675,16.289651,16.324374,...,16.335277,16.372367,16.537346,16.196147,16.173505,16.310830,16.254974,16.348267,16.293014,16.530490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q2QGD7,15.627144,15.481194,15.726177,15.928917,15.457205,15.527411,15.670059,15.073871,14.955910,15.532734,...,15.273730,15.516053,15.838997,15.634068,15.526009,15.371021,15.388584,15.538631,15.627802,15.507224
Q9C0D3,20.918340,20.842082,20.881666,20.944515,21.111598,20.888484,21.184368,20.642628,20.619563,20.920692,...,21.109123,20.985639,20.867963,20.997213,20.730738,20.547531,20.861241,20.549092,20.461376,20.356947
Q15942,28.917052,28.813093,28.692954,29.264926,29.331659,28.669558,28.872298,28.994721,28.989805,28.779167,...,28.397063,28.819882,29.257749,28.014804,27.562795,27.765326,28.010625,27.360692,27.641424,27.304978


In [61]:
reference_intensity_row

,ReferenceIntensity
Sample ID,
P04217,28.716163
Q9NQ94,24.592863
P01023,29.941210
A8K2U0,21.881356
Q9NPC4,16.076463
...,...
Q2QGD7,15.456372
Q9C0D3,20.805448
Q15942,28.865534


In [44]:
# Guardar el dataframe en formato tsv
reference_intensity_row.to_csv('reference_intensity.tsv', sep='\t', index=True)

In [62]:
clases = pd.read_csv('class.csv', sep='\t')
clases

,Class
0,Tumor
1,Tumor
2,Tumor
3,Tumor
4,Tumor
...,...
229,NAT
230,NAT
231,NAT
232,NAT


In [63]:
# Trasposición de los datos
data= data.T
data

Sample ID,P04217,Q9NQ94,P01023,A8K2U0,Q9NPC4,Q9UNA3,Q9NRG9,Q86V21,P22760,Q8N5Z0,...,Q86XI8,O43264,Q9H900,O95229,P98168,Q2QGD7,Q9C0D3,Q15942,O43149,Q8IYH5
CPT0237840004,28.499551,24.018242,29.373080,21.264149,17.221701,19.963645,23.160997,24.105541,23.180064,16.443033,...,18.851081,25.001004,23.743554,17.467196,17.072977,15.627144,20.918340,28.917052,25.079604,19.765872
CPT0166760004,28.452195,23.500280,29.038021,21.538211,16.361668,20.007376,23.368541,24.372142,22.724454,16.086453,...,18.757266,24.976303,23.293711,17.399244,17.253997,15.481194,20.842082,28.813093,24.908095,19.598989
CPT0094630004,28.630157,24.174142,29.764657,21.404347,16.615096,19.890128,22.787457,24.395965,23.358289,15.660907,...,18.517571,24.995980,23.032407,17.282745,17.141744,15.726177,20.881666,28.692954,24.953850,19.639184
CPT0063970004,29.174342,23.972000,30.442977,21.567005,16.679397,19.905289,22.754717,24.154326,22.499465,15.888535,...,18.462373,24.884388,22.772827,17.170056,17.305030,15.928917,20.944515,29.264926,24.950274,19.467798
CPT0081260004,29.585927,24.501145,29.902549,21.898294,17.552648,19.240848,22.866840,23.946312,22.062902,15.635474,...,17.684352,24.259581,22.512645,17.406196,17.630532,15.457205,21.111598,29.331659,24.707046,19.597537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CPT0347760002,28.488643,25.981225,30.546957,20.980295,16.310830,19.650159,22.947035,24.576902,21.975084,14.572037,...,18.594922,25.411646,23.149712,17.027794,17.133707,15.371021,20.547531,27.765326,24.978352,19.697366
CPT0347790002,28.385020,25.599940,29.339427,21.025700,16.254974,19.674408,23.313148,24.212067,20.799857,14.501354,...,18.901277,25.282695,22.777488,16.973744,17.032075,15.388584,20.861241,28.010625,24.781041,19.755829
CPT0347820002,27.729956,25.914578,28.685161,20.925103,16.348267,20.062441,23.220923,24.892807,22.770046,14.880148,...,18.514863,25.709621,22.715885,16.679245,16.929581,15.538631,20.549092,27.360692,25.387207,19.579736
CPT0347850002,28.458926,25.671271,29.364176,21.207741,16.293014,19.854315,23.189772,24.761407,22.694296,14.797793,...,18.304059,25.563213,22.856631,16.878188,16.825199,15.627802,20.461376,27.641424,25.235190,19.517803


In [64]:
data = data.reset_index()
data

Sample ID,index,P04217,Q9NQ94,P01023,A8K2U0,Q9NPC4,Q9UNA3,Q9NRG9,Q86V21,P22760,...,Q86XI8,O43264,Q9H900,O95229,P98168,Q2QGD7,Q9C0D3,Q15942,O43149,Q8IYH5
0,CPT0237840004,28.499551,24.018242,29.373080,21.264149,17.221701,19.963645,23.160997,24.105541,23.180064,...,18.851081,25.001004,23.743554,17.467196,17.072977,15.627144,20.918340,28.917052,25.079604,19.765872
1,CPT0166760004,28.452195,23.500280,29.038021,21.538211,16.361668,20.007376,23.368541,24.372142,22.724454,...,18.757266,24.976303,23.293711,17.399244,17.253997,15.481194,20.842082,28.813093,24.908095,19.598989
2,CPT0094630004,28.630157,24.174142,29.764657,21.404347,16.615096,19.890128,22.787457,24.395965,23.358289,...,18.517571,24.995980,23.032407,17.282745,17.141744,15.726177,20.881666,28.692954,24.953850,19.639184
3,CPT0063970004,29.174342,23.972000,30.442977,21.567005,16.679397,19.905289,22.754717,24.154326,22.499465,...,18.462373,24.884388,22.772827,17.170056,17.305030,15.928917,20.944515,29.264926,24.950274,19.467798
4,CPT0081260004,29.585927,24.501145,29.902549,21.898294,17.552648,19.240848,22.866840,23.946312,22.062902,...,17.684352,24.259581,22.512645,17.406196,17.630532,15.457205,21.111598,29.331659,24.707046,19.597537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,CPT0347760002,28.488643,25.981225,30.546957,20.980295,16.310830,19.650159,22.947035,24.576902,21.975084,...,18.594922,25.411646,23.149712,17.027794,17.133707,15.371021,20.547531,27.765326,24.978352,19.697366
230,CPT0347790002,28.385020,25.599940,29.339427,21.025700,16.254974,19.674408,23.313148,24.212067,20.799857,...,18.901277,25.282695,22.777488,16.973744,17.032075,15.388584,20.861241,28.010625,24.781041,19.755829
231,CPT0347820002,27.729956,25.914578,28.685161,20.925103,16.348267,20.062441,23.220923,24.892807,22.770046,...,18.514863,25.709621,22.715885,16.679245,16.929581,15.538631,20.549092,27.360692,25.387207,19.579736
232,CPT0347850002,28.458926,25.671271,29.364176,21.207741,16.293014,19.854315,23.189772,24.761407,22.694296,...,18.304059,25.563213,22.856631,16.878188,16.825199,15.627802,20.461376,27.641424,25.235190,19.517803


In [65]:
if len(data) == len(clases):
    print('True')

True


In [66]:
# Insertar la columna Class en el dataframe
data.insert(1, 'Class', clases, allow_duplicates=True)
data = data.rename(columns={'index': 'Sample ID'})
data

Sample ID,Sample ID,Class,P04217,Q9NQ94,P01023,A8K2U0,Q9NPC4,Q9UNA3,Q9NRG9,Q86V21,...,Q86XI8,O43264,Q9H900,O95229,P98168,Q2QGD7,Q9C0D3,Q15942,O43149,Q8IYH5
0,CPT0237840004,Tumor,28.499551,24.018242,29.373080,21.264149,17.221701,19.963645,23.160997,24.105541,...,18.851081,25.001004,23.743554,17.467196,17.072977,15.627144,20.918340,28.917052,25.079604,19.765872
1,CPT0166760004,Tumor,28.452195,23.500280,29.038021,21.538211,16.361668,20.007376,23.368541,24.372142,...,18.757266,24.976303,23.293711,17.399244,17.253997,15.481194,20.842082,28.813093,24.908095,19.598989
2,CPT0094630004,Tumor,28.630157,24.174142,29.764657,21.404347,16.615096,19.890128,22.787457,24.395965,...,18.517571,24.995980,23.032407,17.282745,17.141744,15.726177,20.881666,28.692954,24.953850,19.639184
3,CPT0063970004,Tumor,29.174342,23.972000,30.442977,21.567005,16.679397,19.905289,22.754717,24.154326,...,18.462373,24.884388,22.772827,17.170056,17.305030,15.928917,20.944515,29.264926,24.950274,19.467798
4,CPT0081260004,Tumor,29.585927,24.501145,29.902549,21.898294,17.552648,19.240848,22.866840,23.946312,...,17.684352,24.259581,22.512645,17.406196,17.630532,15.457205,21.111598,29.331659,24.707046,19.597537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,CPT0347760002,NAT,28.488643,25.981225,30.546957,20.980295,16.310830,19.650159,22.947035,24.576902,...,18.594922,25.411646,23.149712,17.027794,17.133707,15.371021,20.547531,27.765326,24.978352,19.697366
230,CPT0347790002,NAT,28.385020,25.599940,29.339427,21.025700,16.254974,19.674408,23.313148,24.212067,...,18.901277,25.282695,22.777488,16.973744,17.032075,15.388584,20.861241,28.010625,24.781041,19.755829
231,CPT0347820002,NAT,27.729956,25.914578,28.685161,20.925103,16.348267,20.062441,23.220923,24.892807,...,18.514863,25.709621,22.715885,16.679245,16.929581,15.538631,20.549092,27.360692,25.387207,19.579736
232,CPT0347850002,NAT,28.458926,25.671271,29.364176,21.207741,16.293014,19.854315,23.189772,24.761407,...,18.304059,25.563213,22.856631,16.878188,16.825199,15.627802,20.461376,27.641424,25.235190,19.517803


In [42]:
# Guardar el dataframe en formato tsv
data.to_csv('ion_abundance_data.tsv', sep='\t', index=False)

### SELECCIONAR DE LA MATRIZ DE DATOS "DATA" LAS 41 MUESTRAS TUMORALES Y NORMALES PAREADAS PARA EL ANÁLISIS

In [67]:
# Lista de Sample IDs que quieres seleccionar
sample_ids_to_select = ["CPT0063580003", "CPT0063570003", "CPT0025760003", "CPT0025750003", "CPT0237840004", "CPT0237850004", "CPT0124980004", "CPT0125010003", "CPT0127290003", "CPT0127270003", "CPT0011520003", "CPT0011540003", "CPT0183450005", "C3N-02573-03", "CPT0077860003", "CPT0077880003", "CPT0198420004", "CPT0198440004", "CPT0203850004", "C3N-00709-06", "CPT0236700005", "CPT0236710004", "CPT0208710004", "CPT0208690004", "CPT0170260004", "CPT0170240004", "CPT0237910004", "CPT0237920004", "CPT0162780004", "CPT0162760003", "CPT0091680003", "CPT0091660003", "CPT0198930003", "CPT0198910005", "CPT0001730017", "CPT0001750003", "CPT0162670004", "CPT0162660004", "CPT0238640004", "CPT0238610009", "CPT0064090003", "CPT0064110003", "CPT0183770004", "CPT0183750005", "CPT0218200004", "CPT0218220004", "CPT0208840004", "CPT0208850004", "CPT0239920004", "CPT0239910004", "CPT0238940004", "CPT0238920004", "CPT0221250004", "CPT0221240004", "CPT0166780004", "CPT0166760004", "CPT0246990003", "CPT0247000003", "CPT0123650003", "CPT0123670003", "CPT0241500004", "CPT0241510004", "CPT0197420004", "CPT0197440004", "CPT0086190004", "CPT0086210003", "CPT0236780005", "CPT0236800004", "CPT0226600004", "CPT0226610004", "CPT0174800003", "CPT0174700004", "CPT0248410004", "CPT0248440003", "CPT0246940003", "CPT0246950003", "CPT0078000003", "CPT0078030003", "CPT0108930003", "CPT0108960003", "CPT0248590004", "CPT0248620003"]

# Seleccionar las filas con los Sample IDs dados
data_41 = data.loc[data['Sample ID'].isin(sample_ids_to_select)]
data_41

Sample ID,Sample ID,Class,P04217,Q9NQ94,P01023,A8K2U0,Q9NPC4,Q9UNA3,Q9NRG9,Q86V21,...,Q86XI8,O43264,Q9H900,O95229,P98168,Q2QGD7,Q9C0D3,Q15942,O43149,Q8IYH5
0,CPT0237840004,Tumor,28.499551,24.018242,29.373080,21.264149,17.221701,19.963645,23.160997,24.105541,...,18.851081,25.001004,23.743554,17.467196,17.072977,15.627144,20.918340,28.917052,25.079604,19.765872
1,CPT0166760004,Tumor,28.452195,23.500280,29.038021,21.538211,16.361668,20.007376,23.368541,24.372142,...,18.757266,24.976303,23.293711,17.399244,17.253997,15.481194,20.842082,28.813093,24.908095,19.598989
7,CPT0198420004,Tumor,28.876948,23.584650,30.854922,21.086050,16.191675,19.375648,23.109027,23.656449,...,19.115999,24.973140,23.106868,16.918797,17.015573,15.073871,20.642628,28.994721,24.686197,19.802472
8,CPT0198910005,Tumor,28.352665,23.575798,29.868366,21.337391,16.289651,19.823537,23.058595,24.438543,...,18.676713,25.083902,23.201727,16.853149,16.743348,14.955910,20.619563,28.989805,24.843689,19.740435
12,CPT0124980004,Tumor,28.412826,23.669284,29.539075,20.882927,16.167757,19.225588,23.621645,24.178937,...,18.334897,24.846971,22.563097,17.542299,17.422285,14.914517,21.221403,29.254572,24.431228,17.689143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,C3N-02573-03,NAT,28.313648,26.588224,29.957278,21.130595,16.223742,19.397507,23.109687,23.922125,...,18.514517,25.542430,23.684894,17.083855,17.078059,15.246541,20.881919,28.133745,24.889034,20.182123
216,C3N-00709-06,NAT,28.368991,26.109642,30.356308,21.201422,16.328382,20.215519,23.008220,24.365625,...,18.059888,25.519741,23.475605,17.818606,17.268339,15.267254,21.102749,28.081646,25.055802,19.914253
217,CPT0127290003,NAT,28.250506,26.158056,29.508487,21.288262,15.943571,20.029094,22.906250,24.149753,...,18.906324,25.310035,22.914869,16.945714,17.436270,16.136545,20.855694,28.075227,25.001542,20.019647
220,CPT0236710004,NAT,28.273358,25.750190,28.718255,20.381522,15.989643,19.944925,22.794078,24.417816,...,18.164556,25.240468,23.641317,17.248901,15.819356,13.722731,21.166533,28.312253,25.334794,20.518658


In [68]:
# Guardar el dataframe en formato tsv
data_41.to_csv('ion_abundance_41.tsv', sep='\t', index=False)